In [4]:
import numpy as np
import h5py
import glob, json
import sys
sys.path.append('./../code/')
from support import *
from likelihoods import *

Load successfully found injections, which we will reweight to our proposed population fits in preparation for predictive checks

In [2]:
# Load mock detections
mMin = 5.

mockDetections = h5py.File('../input/o3a_bbhpop_inj_info.hdf','r')
ifar_1 = mockDetections['injections']['ifar_gstlal'].value
ifar_2 = mockDetections['injections']['ifar_pycbc_bbh'].value
ifar_3 = mockDetections['injections']['ifar_pycbc_full'].value
detected = (ifar_1>1) + (ifar_2>1) + (ifar_3>1)
m1_det = mockDetections['injections']['mass1_source'].value[detected]
m2_det = mockDetections['injections']['mass2_source'].value[detected]
s1z_det = mockDetections['injections']['spin1z'].value[detected]
s2z_det = mockDetections['injections']['spin2z'].value[detected]
z_det = mockDetections['injections']['redshift'].value[detected]

mockDetectionsO1O2 = h5py.File('../input/injections_O1O2an_spin.h5','r')
m1_det = np.append(m1_det,mockDetectionsO1O2['mass1_source'])
m2_det = np.append(m2_det,mockDetectionsO1O2['mass2_source'])
s1z_det = np.append(s1z_det,mockDetectionsO1O2['spin1z'])
s2z_det = np.append(s2z_det,mockDetectionsO1O2['spin2z'])
z_det = np.append(z_det,mockDetectionsO1O2['redshift'])

# Derived quantities
q_det = m2_det/m1_det
X_det = (m1_det*s1z_det + m2_det*s2z_det)/(m1_det+m2_det)

pop_reweight = injection_weights(m1_det,m2_det,s1z_det,s2z_det,z_det,mMin=5.)

/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/ipykernel/__main__.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/ipykernel/__main__.py:6: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/ipykernel/__main__.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/ipykernel/__main__.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py37/lib/python3.7/site-packages/ipykernel/__main__.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
/cvm

# 1. Population with no mass ratio-spin correlations

In [9]:
n_catalogs = 750
posteriors = np.load("../input/sampleDict.pickle",allow_pickle=True)
samps_no_evol = np.load('../code/output/processed_emcee_samples_plPeak_noEvol_r00.npy')

# Prepare arrays to hold mock draws from injection sets and reweighted posteriors
mock_q_noEvol = np.zeros((len(posteriors),n_catalogs))
mock_x_noEvol = np.zeros((len(posteriors),n_catalogs))
resampled_q_noEvol = np.zeros((len(posteriors),n_catalogs))
resampled_x_noEvol = np.zeros((len(posteriors),n_catalogs))

mu_chi = np.zeros(n_catalogs)
logsig_chi = np.zeros(n_catalogs)

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps_no_evol.shape[0]))
    
    mMin = 5.
    lmbda = samps_no_evol[samp,0]
    mMax = samps_no_evol[samp,1]
    m0 = samps_no_evol[samp,2]
    sigM = samps_no_evol[samp,3]
    fPeak = samps_no_evol[samp,4]
    bq = samps_no_evol[samp,5]
    kappa = samps_no_evol[samp,6]
    mu0 = samps_no_evol[samp,7]
    logsig0 = samps_no_evol[samp,8]
    
    mu_chi[i] = mu0
    logsig_chi[i] = logsig0
    
    # New p(m1)
    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl
    
    # New p(m2)
    mock_p_m2 = (1.+bq)*np.power(m2_det,bq)/(np.power(m1_det,1.+bq)-mMin**(1.+bq))
    mock_p_m2[m2_det<mMin]=0
    
    # New p(z) and p(chi)
    mock_p_z = np.power(1.+z_det,kappa-1.)
    mock_p_chi = calculate_Gaussian(X_det, mu0, 10.**(2.*logsig0),-1.,1.)
    
    p_det = mock_p_m1*mock_p_m2*mock_p_z*mock_p_chi*pop_reweight
    p_det /= np.sum(p_det)
    
    # Given detection weights, draw a mock event
    try:
        detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors),p=p_det,replace=True)
        
    except ValueError:
        print(samp)
        
    # Save selected value
    mock_q_noEvol[:,i] = q_det[detected_injections]
    mock_x_noEvol[:,i] = X_det[detected_injections]

    # Now loop across events
    for ii,key in enumerate(posteriors):

        # Read out properties
        chis = posteriors[key]['Xeff']
        Xeff_prior = posteriors[key]['Xeff_priors']
        m1s = posteriors[key]['m1']
        m2s = posteriors[key]['m2']
        zs = posteriors[key]['z']
        weights = posteriors[key]['weights']
        qs = m2s/m1s

        p_Chi = calculate_Gaussian(chis, mu0, 10.**(2.*logsig0),-1.,1.)

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)

        # p(m2)
        p_m2 = (1.+bq)*np.power(m2s,bq)/(np.power(m1s,1.+bq)-mMin**(1.+bq))
        old_m2_prior = np.ones(m1s.size)
        p_m2[m2s<mMin]=0

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)
        
        # Draw probs
        probs = p_Chi*p_m1*p_m2*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior
        
        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)
        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        
        # Save selected sample
        resampled_q_noEvol[ii,i] = qs[chosenInd]
        resampled_x_noEvol[ii,i] = chis[chosenInd]

In [10]:
reweighted_sample_dict = {
    'mock_q':mock_q_noEvol,
    'mock_chi':mock_x_noEvol,
    'resampled_q':resampled_q_noEvol,
    'resampled_chi':resampled_x_noEvol
}

np.save('./reweighted_samples_noEvolution.npy',reweighted_sample_dict)

# 2. Population *with* chi-q correlations

In [3]:
n_catalogs = 750
posteriors = np.load("../input/sampleDict.pickle",allow_pickle=True)
samps = np.load('../code/output/processed_emcee_samples_plPeak_r00.npy')

mock_q = np.zeros((len(posteriors),n_catalogs))
mock_x = np.zeros((len(posteriors),n_catalogs))

resampled_q = np.zeros((len(posteriors),n_catalogs))
resampled_x = np.zeros((len(posteriors),n_catalogs))

mu_chi_evol = np.zeros(n_catalogs)
logsig_chi_evol = np.zeros(n_catalogs)
alpha_evol = np.zeros(n_catalogs)
beta_evol = np.zeros(n_catalogs)

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps.shape[0]))
    
    mMin = 5.
    lmbda = samps[samp,0]
    mMax = samps[samp,1]
    m0 = samps[samp,2]
    sigM = samps[samp,3]
    fPeak = samps[samp,4]
    bq = samps[samp,5]
    kappa = samps[samp,6]
    mu0 = samps[samp,7]
    logsig0 = samps[samp,8]
    alpha = samps[samp,9]
    beta = samps[samp,10]
    
    mu_chi_evol[i] = mu0
    logsig_chi_evol[i] = logsig0    
    alpha_evol[i] = alpha
    beta_evol[i] = beta
    
    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl 
    
    mock_p_m2 = (1.+bq)*np.power(m2_det,bq)/(np.power(m1_det,1.+bq)-mMin**(1.+bq))
    mock_p_m2[m2_det<mMin]=0
    
    mock_p_z = np.power(1.+z_det,kappa-1.)
    
    mu = mu0 + alpha*(q_det-0.5)
    logsig = logsig0 + beta*(q_det-0.5)
    mock_p_chi = calculate_Gaussian(X_det, mu, 10.**(2.*logsig),-1.,1.)
    
    p_det = mock_p_m1*mock_p_m2*mock_p_z*mock_p_chi*pop_reweight
    p_det /= np.sum(p_det)
    detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors),p=p_det,replace=False)
    mock_q[:,i] = q_det[detected_injections]
    mock_x[:,i] = X_det[detected_injections]

    for ii,key in enumerate(posteriors):

        chis = posteriors[key]['Xeff']
        Xeff_prior = posteriors[key]['Xeff_priors']
        m1s = posteriors[key]['m1']
        m2s = posteriors[key]['m2']
        zs = posteriors[key]['z']
        weights = posteriors[key]['weights']
        qs = m2s/m1s
        
        mu = mu0 + alpha*(qs-0.5)
        logsig = logsig0 + beta*(qs-0.5)
        p_Chi = calculate_Gaussian(chis, mu, 10.**(2.*logsig),-1.,1.)

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)

        # p(m2)
        p_m2 = (1.+bq)*np.power(m2s,bq)/(np.power(m1s,1.+bq)-mMin**(1.+bq))
        old_m2_prior = np.ones(m1s.size)
        p_m2[m2s<mMin]=0

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)
        
        # Draw probs
        probs = p_Chi*p_m1*p_m2*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior
        
        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)
        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        
        resampled_q[ii,i] = qs[chosenInd]
        resampled_x[ii,i] = chis[chosenInd]

In [4]:
reweighted_sample_dict = {
    'mock_q':mock_q,
    'mock_chi':mock_x,
    'resampled_q':resampled_q,
    'resampled_chi':resampled_x
}

np.save('./reweighted_samples_yesEvolution.npy',reweighted_sample_dict)

# 3. Population with spin-q correlations, neglecting GW190412

In [13]:
n_catalogs = 750
posteriors = np.load("../input/sampleDict.pickle",allow_pickle=True)
posteriors.pop('S190412m')
samps_no190412 = np.load('../code/output/processed_emcee_samples_plPeak_no190412_r00.npy')

mock_q_no190412 = np.zeros((len(posteriors),n_catalogs))
mock_x_no190412 = np.zeros((len(posteriors),n_catalogs))

resampled_q_no190412 = np.zeros((len(posteriors),n_catalogs))
resampled_x_no190412 = np.zeros((len(posteriors),n_catalogs))

mu_chi_evol_no190412 = np.zeros(n_catalogs)
logsig_chi_evol_no190412 = np.zeros(n_catalogs)
alpha_evol_no190412 = np.zeros(n_catalogs)
beta_evol_no190412 = np.zeros(n_catalogs)

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps_no190412.shape[0]))

    mMin = 5.
    lmbda = samps_no190412[samp,0]
    mMax = samps_no190412[samp,1]
    m0 = samps_no190412[samp,2]
    sigM = samps_no190412[samp,3]
    fPeak = samps_no190412[samp,4]
    bq = samps_no190412[samp,5]
    kappa = samps_no190412[samp,6]
    mu0 = samps_no190412[samp,7]
    logsig0 = samps_no190412[samp,8]
    alpha = samps_no190412[samp,9]
    beta = samps_no190412[samp,10]

    mu_chi_evol_no190412[i] = mu0
    logsig_chi_evol_no190412[i] = logsig0    
    alpha_evol_no190412[i] = alpha
    beta_evol_no190412[i] = beta

    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl  

    mock_p_m2 = (1.+bq)*np.power(m2_det,bq)/(np.power(m1_det,1.+bq)-mMin**(1.+bq))
    mock_p_m2[m2_det<mMin]=0

    mock_p_z = np.power(1.+z_det,kappa-1.)

    mu = mu0 + alpha*(q_det-0.5)
    logsig = logsig0 + beta*(q_det-0.5)
    mock_p_chi = calculate_Gaussian(X_det, mu, 10.**(2.*logsig),-1.,1.)

    p_det = mock_p_m1*mock_p_m2*mock_p_z*mock_p_chi*pop_reweight
    p_det /= np.sum(p_det)

    detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors),p=p_det,replace=False)
    mock_q_no190412[:,i] = q_det[detected_injections]
    mock_x_no190412[:,i] = X_det[detected_injections]

    for ii,key in enumerate(posteriors):

        chis = posteriors[key]['Xeff']
        Xeff_prior = posteriors[key]['Xeff_priors']
        m1s = posteriors[key]['m1']
        m2s = posteriors[key]['m2']
        zs = posteriors[key]['z']
        weights = posteriors[key]['weights']
        qs = m2s/m1s

        mu = mu0 + alpha*(qs-0.5)
        logsig = logsig0 + beta*(qs-0.5)
        p_Chi = calculate_Gaussian(chis, mu, 10.**(2.*logsig),-1.,1.)

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)

        # p(m2)
        p_m2 = (1.+bq)*np.power(m2s,bq)/(np.power(m1s,1.+bq)-mMin**(1.+bq))
        old_m2_prior = np.ones(m1s.size)
        p_m2[m2s<mMin]=0

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)

        # Draw probs
        probs = p_Chi*p_m1*p_m2*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior

        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)

        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        resampled_q_no190412[ii,i] = qs[chosenInd]
        resampled_x_no190412[ii,i] = chis[chosenInd]

In [14]:
reweighted_sample_dict = {
    'mock_q':mock_q_no190412,
    'mock_chi':mock_x_no190412,
    'resampled_q':resampled_q_no190412,
    'resampled_chi':resampled_x_no190412
}

np.save('./reweighted_samples_yesEvolution_no190412.npy',reweighted_sample_dict)

# 4. Correlated population including GW190814

In [16]:
n_catalogs = 750
posteriors_w190814 = np.load("../input/sampleDict_w190814.pickle",allow_pickle=True)
samps_w190814 = np.load('../code/output/processed_emcee_samples_plPeak_w190814_r00.npy')

mock_q_w190814 = np.zeros((len(posteriors_w190814),n_catalogs))
mock_x_w190814 = np.zeros((len(posteriors_w190814),n_catalogs))

resampled_q_w190814 = np.zeros((len(posteriors_w190814),n_catalogs))
resampled_x_w190814 = np.zeros((len(posteriors_w190814),n_catalogs))

mu_chi_evol_w190814 = np.zeros(n_catalogs)
logsig_chi_evol_w190814 = np.zeros(n_catalogs)
alpha_evol_w190814 = np.zeros(n_catalogs)
beta_evol_w190814 = np.zeros(n_catalogs)

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps_w190814.shape[0]))
    
    mMin = 2.5
    lmbda = samps_w190814[samp,0]
    mMax = samps_w190814[samp,1]
    m0 = samps_w190814[samp,2]
    sigM = samps_w190814[samp,3]
    fPeak = samps_w190814[samp,4]
    bq = samps_w190814[samp,5]
    kappa = samps_w190814[samp,6]
    mu0 = samps_w190814[samp,7]
    logsig0 = samps_w190814[samp,8]
    alpha = samps_w190814[samp,9]
    beta = samps_w190814[samp,10]
    
    mu_chi_evol_w190814[i] = mu0
    logsig_chi_evol_w190814[i] = logsig0    
    alpha_evol_w190814[i] = alpha
    beta_evol_w190814[i] = beta
    
    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl  
    
    mock_p_m2 = (1.+bq)*np.power(m2_det,bq)/(np.power(m1_det,1.+bq)-mMin**(1.+bq))
    mock_p_m2[m2_det<mMin]=0
    
    mock_p_z = np.power(1.+z_det,kappa-1.)
    
    mu = mu0 + alpha*(q_det-0.5)
    logsig = logsig0 + beta*(q_det-0.5)
    mock_p_chi = calculate_Gaussian(X_det, mu, 10.**(2.*logsig),-1.,1.)
    
    p_det = mock_p_m1*mock_p_m2*mock_p_z*mock_p_chi*pop_reweight
    p_det /= np.sum(p_det)
    detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors_w190814),p=p_det,replace=False)
    mock_q_w190814[:,i] = q_det[detected_injections]
    mock_x_w190814[:,i] = X_det[detected_injections]

    for ii,key in enumerate(posteriors_w190814):

        chis = posteriors_w190814[key]['Xeff']
        Xeff_prior = posteriors_w190814[key]['Xeff_priors']
        m1s = posteriors_w190814[key]['m1']
        m2s = posteriors_w190814[key]['m2']
        zs = posteriors_w190814[key]['z']
        weights = posteriors_w190814[key]['weights']
        qs = m2s/m1s
        
        mu = mu0 + alpha*(qs-0.5)
        logsig = logsig0 + beta*(qs-0.5)
        p_Chi = calculate_Gaussian(chis, mu, 10.**(2.*logsig),-1.,1.)

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)

        # p(m2)
        p_m2 = (1.+bq)*np.power(m2s,bq)/(np.power(m1s,1.+bq)-mMin**(1.+bq))
        old_m2_prior = np.ones(m1s.size)
        p_m2[m2s<mMin]=0

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)
        
        # Draw probs
        probs = p_Chi*p_m1*p_m2*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior
        
        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)
        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        
        resampled_q_w190814[ii,i] = qs[chosenInd]
        resampled_x_w190814[ii,i] = chis[chosenInd]

In [17]:
reweighted_sample_dict = {
    'mock_q':mock_q_w190814,
    'mock_chi':mock_x_w190814,
    'resampled_q':resampled_q_w190814,
    'resampled_chi':resampled_x_w190814
}

np.save('./reweighted_samples_yesEvolution_w190814.npy',reweighted_sample_dict)

# 5. Correlated population with Gaussian p(q|m1)

In [19]:
n_catalogs = 750
posteriors = np.load("../input/sampleDict.pickle",allow_pickle=True)
samps_gaussianQ = np.load('./../code/output/processed_emcee_samples_plPeak_gaussianQ_r00.npy')

mock_q_gaussianQ = np.zeros((len(posteriors),n_catalogs))
mock_x_gaussianQ = np.zeros((len(posteriors),n_catalogs))
resampled_q_gaussianQ = np.zeros((len(posteriors),n_catalogs))
resampled_x_gaussianQ = np.zeros((len(posteriors),n_catalogs))

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps_gaussianQ.shape[0]))
    
    mMin = 5.
    lmbda = samps_gaussianQ[samp,0]
    mMax = samps_gaussianQ[samp,1]
    m0 = samps_gaussianQ[samp,2]
    sigM = samps_gaussianQ[samp,3]
    fPeak = samps_gaussianQ[samp,4]
    mu_q = samps_gaussianQ[samp,5]
    sig_q = samps_gaussianQ[samp,6]
    kappa = samps_gaussianQ[samp,7]
    mu0 = samps_gaussianQ[samp,8]
    logsig0 = samps_gaussianQ[samp,9]
    alpha = samps_gaussianQ[samp,10]
    beta = samps_gaussianQ[samp,11]
    
    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl  
    
    mock_p_m2 = calculate_Gaussian(q_det,mu_q,sig_q,mMin/m1_det,1.)/m1_det
    mock_p_m2[m2_det<mMin]=0
    
    mock_p_z = np.power(1.+z_det,kappa-1.)
    
    mu = mu0 + alpha*(q_det-0.5)
    logsig = logsig0 + beta*(q_det-0.5)
    mock_p_chi = calculate_Gaussian(X_det, mu, 10.**(2.*logsig),-1.,1.)
    
    p_det = mock_p_m1*mock_p_m2*mock_p_z*mock_p_chi*pop_reweight
    p_det /= np.sum(p_det)
    detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors),p=p_det,replace=False)
    mock_q_gaussianQ[:,i] = q_det[detected_injections]
    mock_x_gaussianQ[:,i] = X_det[detected_injections]

    for ii,key in enumerate(posteriors):

        chis = posteriors[key]['Xeff']
        Xeff_prior = posteriors[key]['Xeff_priors']
        m1s = posteriors[key]['m1']
        m2s = posteriors[key]['m2']
        zs = posteriors[key]['z']
        weights = posteriors[key]['weights']
        qs = m2s/m1s
        
        mu = mu0 + alpha*(qs-0.5)
        logsig = logsig0 + beta*(qs-0.5)
        p_Chi = calculate_Gaussian(chis, mu, 10.**(2.*logsig),-1.,1.)

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)

        # p(m2)
        p_m2 = calculate_Gaussian(qs,mu_q,sig_q,mMin/m1s,1.)/m1s
        old_m2_prior = np.ones(m1s.size)
        p_m2[m2s<mMin]=0

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)
        
        # Draw probs
        probs = p_Chi*p_m1*p_m2*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior
        
        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)
        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        
        resampled_q_gaussianQ[ii,i] = qs[chosenInd]
        resampled_x_gaussianQ[ii,i] = chis[chosenInd]

In [20]:
reweighted_sample_dict = {
    'mock_q':mock_q_gaussianQ,
    'mock_chi':mock_x_gaussianQ,
    'resampled_q':resampled_q_gaussianQ,
    'resampled_chi':resampled_x_gaussianQ
}

np.save('./reweighted_samples_yesEvolution_gaussianQ.npy',reweighted_sample_dict)

# 6. Mixture of two bivariate Gaussians

In [ ]:
n_catalogs = 750
posteriors = np.load("../input/sampleDict.pickle",allow_pickle=True)
samps_bivariateGaussian = np.load('./../code/output/processed_emcee_samples_plPeak_bivariateGaussian_r05.npy')

mock_q_bivariateGaussian = np.zeros((len(posteriors),n_catalogs))
mock_x_bivariateGaussian = np.zeros((len(posteriors),n_catalogs))
resampled_q_bivariateGaussian = np.zeros((len(posteriors),400))
resampled_x_bivariateGaussian = np.zeros((len(posteriors),400))

for i in range(n_catalogs):
    
    samp = np.random.choice(np.arange(samps_bivariateGaussian.shape[0]))
    
    mMin = 5.
    lmbda = samps_bivariateGaussian[samp,0]
    mMax = samps_bivariateGaussian[samp,1]
    m0 = samps_bivariateGaussian[samp,2]
    sigM = samps_bivariateGaussian[samp,3]
    fPeak = samps_bivariateGaussian[samp,4]
    f_lowSpin = samps_bivariateGaussian[samp,5]
    mu_q_lowSpin = samps_bivariateGaussian[samp,6]
    logsig_q_lowSpin = samps_bivariateGaussian[samp,7]
    mu_q_highSpin = samps_bivariateGaussian[samp,8]
    logsig_q_highSpin = samps_bivariateGaussian[samp,9]
    kappa = samps_bivariateGaussian[samp,10]
    mu_chi_lowSpin = samps_bivariateGaussian[samp,11]
    logsig_chi_lowSpin = samps_bivariateGaussian[samp,12]
    mu_chi_highSpin = samps_bivariateGaussian[samp,13]
    logsig_chi_highSpin = samps_bivariateGaussian[samp,14]

    # p(m1)
    p_det_m1_pl = (1.+lmbda)*m1_det**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
    p_det_m1_pl[m1_det>mMax] = 0.
    p_det_m1_peak = np.exp(-0.5*(m1_det-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
    mock_p_m1 = fPeak*p_det_m1_peak + (1.-fPeak)*p_det_m1_pl

    # p(m2)
    mock_p_Xeff_m2_lowSpin = calculate_Gaussian_2D(X_det,q_det,mu_chi_lowSpin,10.**(2.*logsig_chi_lowSpin),\
                mu_q_lowSpin,10.**(2.*logsig_q_lowSpin),-1.,1.,0.,1.)/m1_det
    mock_p_Xeff_m2_highSpin = calculate_Gaussian_2D(X_det,q_det,mu_chi_highSpin,10.**(2.*logsig_chi_highSpin),\
                mu_q_highSpin,10.**(2.*logsig_q_highSpin),-1.,1.,0.,1.)/m1_det
    mock_p_Xeff_m2 = f_lowSpin*mock_p_Xeff_m2_lowSpin + (1.-f_lowSpin)*mock_p_Xeff_m2_highSpin
    mock_p_Xeff_m2[m2_det<mMin] = 0.

    # p(z)
    p_z = np.power(1.+z_det,kappa-1.)
    
    p_det = mock_p_m1*mock_p_Xeff_m2*mock_p_z*pop_reweight
    p_det /= np.sum(p_det)
    detected_injections = np.random.choice(np.arange(m1_det.size),size=len(posteriors),p=p_det,replace=False)
    mock_q_bivariateGaussian[:,i] = q_det[detected_injections]
    mock_x_bivariateGaussian[:,i] = X_det[detected_injections]

    for ii,key in enumerate(posteriors):

        chis = posteriors[key]['Xeff']
        Xeff_prior = posteriors[key]['Xeff_priors']
        m1s = posteriors[key]['m1']
        m2s = posteriors[key]['m2']
        zs = posteriors[key]['z']
        weights = posteriors[key]['weights']
        qs = m2s/m1s

        # p(m1)
        p_m1_pl = (1.+lmbda)*m1s**lmbda/(mMax**(1.+lmbda) - mMin**(1.+lmbda))
        p_m1_pl[m1s>mMax] = 0.
        p_m1_peak = np.exp(-0.5*(m1s-m0)**2./sigM**2)/np.sqrt(2.*np.pi*sigM**2.)
        p_m1 = fPeak*p_m1_peak + (1.-fPeak)*p_m1_pl
        old_m1_prior = np.ones(m1s.size)
        old_m1_prior = np.ones(m2s.size)

        # p(m2)
        p_Xeff_m2_lowSpin = calculate_Gaussian_2D(chis,qs,mu_chi_lowSpin,10.**(2.*logsig_chi_lowSpin),\
                    mu_q_lowSpin,10.**(2.*logsig_q_lowSpin),-1.,1.,0.,1.)/m1s
        p_Xeff_m2_highSpin = calculate_Gaussian_2D(chis,qs,mu_chi_highSpin,10.**(2.*logsig_chi_highSpin),\
                    mu_q_highSpin,10.**(2.*logsig_q_highSpin),-1.,1.,0.,1.)/m1s
        p_Xeff_m2 = f_lowSpin*p_Xeff_m2_lowSpin + (1.-f_lowSpin)*p_Xeff_m2_highSpin
        p_Xeff_m2[m2s<mMin] = 0.

        # p(z)
        p_z = np.power(1.+zs,kappa-1.)
        old_pz_prior = (1.+zs)**(2.7-1.)
        
        # Draw probs
        probs = p_Xeff_m2*p_m1*p_z*weights/Xeff_prior/old_m1_prior/old_m2_prior/old_pz_prior
        
        if np.any(probs!=probs):
            print(np.where(probs!=probs))
            print(probs)
        probs[probs<0] = 0.
        probs /= np.sum(probs)
        chosenInd = np.random.choice(np.arange(m1s.size),p=probs)
        resampled_q_bivariateGaussian[ii,i] = qs[chosenInd]
        resampled_x_bivariateGaussian[ii,i] = chis[chosenInd]

In [ ]:
reweighted_sample_dict = {
    'mock_q':mock_q_bivariateGaussian,
    'mock_chi':mock_x_bivariateGaussian,
    'resampled_q':resampled_q_bivariateGaussian,
    'resampled_chi':resampled_x_bivariateGaussian
}

np.save('./reweighted_samples_yesEvolution_bivariateGaussian.npy',reweighted_sample_dict)

# 7. Injection study results

In [2]:
# Load population samples from injected population
injection_samps_plPeak = np.load('../injection-study/processed_emcee_samples_injection_plPeak_r02.npy')
n_catalogs = 750

In [5]:
# Define a function to reweight to an isotropic prior on s1z and s2z, rather than an aligned prior
# This is done for consistency with the GWTC-2 PE samples
def isotropic_pz(szs,aMax):
    return np.log(aMax/np.abs(szs))/(2.*aMax)

reweightedSampleDictInjected = {}
bilby_output_files = np.sort(glob.glob('../injection-study/output/job_?????_result.json'))
for i,f in enumerate(bilby_output_files):
    
    with open(f,'r') as jf:
        result = json.load(jf)

    m1 = np.array(result['posterior']['content']['mass_1_source'])
    m2 = np.array(result['posterior']['content']['mass_2_source'])
    s1 = np.array(result['posterior']['content']['spin_1z'])
    s2 = np.array(result['posterior']['content']['spin_2z'])
    
    qs = m2/m1
    chis = (m1*s1 + m2*s2)/(m1+m2)
    
    p_szs = isotropic_pz(s1,1.)*isotropic_pz(s2,1.)
    draw_probs = p_szs/np.sum(p_szs)
    
    chosenInds = np.random.choice(np.arange(qs.size),size=2000,replace=True,p=draw_probs)
    qs = qs[chosenInds]
    chis = chis[chosenInds]
    
    reweightedSampleDictInjected[i] = {'q':qs,'x':chis}
    
    print(f)
    print(1./np.max(draw_probs))

../injection-study/output/job_02121_result.json
1728.9184110943913
../injection-study/output/job_02405_result.json
3653.1037427590154
../injection-study/output/job_02688_result.json
1481.7538133372257
../injection-study/output/job_03611_result.json
3110.364279252653
../injection-study/output/job_04003_result.json
1727.5004377801029
../injection-study/output/job_04158_result.json
1856.9409223072234
../injection-study/output/job_04546_result.json
2830.891522314026
../injection-study/output/job_06677_result.json
2655.8450435268924
../injection-study/output/job_07289_result.json
1401.2150662585702
../injection-study/output/job_07842_result.json
1671.051986194712
../injection-study/output/job_08792_result.json
1951.7784121140905
../injection-study/output/job_08909_result.json
838.7770190479383
../injection-study/output/job_08918_result.json
2430.6463367490737
../injection-study/output/job_09738_result.json
1574.1616526296534
../injection-study/output/job_11231_result.json
1834.153418709552


In [6]:
np.save("injection_samples_reweightedToIsotropy.npy",reweightedSampleDictInjected)